# Kruskal Tensors
```
Copyright 2025 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

Kruskal format is a decomposition of a tensor $\mathcal{X}$ as the sum of the outer products as the columns of matrices. For example, we might write:

$
\mathcal{X} = \sum_{r} a_r \circ b_r \circ c_r
$

where a subscript denotes column index and a circle denotes outer product. In other words, the tensor $\mathcal{X}$ is built from the columns of the matrices $A$, $B$, and $C$. It's often helpful to explicitly specify a weight for each outer product, which we do here:

$
\mathcal{X} = \sum_{r} \lambda_r \, a_r \circ b_r \circ c_r
$
The `ktensor` class stores the components of the tensor $\mathcal{X}$ and can perform many operations, e.g., `ttm`, without explicitly forming the tensor $\mathcal{X}$.
## Kruskal tensor format via `ktensor`
Kruskal format stores a tensor as a sum of rank-1 outer products. For example, consider a tensor of the following form:

$
\mathcal{X} = a_1 \circ b_1 \circ c_1 + a_2 \circ b_2 \circ c_2
$

This can be stored in Kruskal form as follows.

In [ ]:
from __future__ import annotations

import numpy as np

import pyttb as ttb

In [ ]:
np.random.seed(0)
A = np.random.rand(4, 2)  # First column is a_1, second is a_2.
B = np.random.rand(3, 2)  # Likewise for B.
C = np.random.rand(2, 2)  # Likewise for C.
X = ttb.ktensor([A, B, C])  # Create the ktensor.
X

In [ ]:
Y = ttb.ktensor(
    [np.random.rand(4, 1), np.random.rand(2, 1), np.random.rand(3, 1)]
)  # Another ktensor.
Y

For Kruskal format, there can be any number of matrices, but every matrix must have the same number of columns. The number of rows can vary.

## Specifying weights in a `ktensor`
Weights for each rank-1 tensor can be specified by passing in a column vector. For example: 

$
\mathcal{X} = \lambda_1 \, a_1 \circ b_1 \circ c_1 + \lambda_2 \, a_2 \circ b_2 \circ c_2
$

In [ ]:
# Upcoming ktensors will be generated with this same initialization.
def generate_sample_ktensor() -> ttb.ktensor:
    np.random.seed(0)
    A = np.random.rand(4, 2)  # Create some data.
    B = np.random.rand(3, 2)
    C = np.random.rand(2, 2)
    weights = np.array([5.0, 0.25])

    return ttb.ktensor([A, B, C], weights)  # Create the ktensor.

In [ ]:
X = generate_sample_ktensor()
X

## Creating a one-dimensional `ktensor`

In [ ]:
np.random.seed(0)
Y = ttb.ktensor([np.random.rand(4, 5)])  # A one-dimensional ktensor.
Y

## Constituent parts of a `ktensor`

In [ ]:
X = generate_sample_ktensor()
X.weights  # Weights or multipliers.

In [ ]:
X.factor_matrices  # Cell array of matrices.

## Creating a `ktensor` from its constituent parts

In [ ]:
X = generate_sample_ktensor()
Y = ttb.ktensor(X.factor_matrices, X.weights)  # Recreate X.
Y

## Creating an empty `ktensor`

In [ ]:
Z = ttb.ktensor()  # Empty ktensor.
Z

## Use `full` or `to_tensor` to convert a `ktensor` to a `tensor`

In [ ]:
X = generate_sample_ktensor()
X.full()  # Converts to a tensor.

In [ ]:
X.to_tensor()  # Same as above.

## Use `double` to convert a `ktensor` to a multidimensional array

In [ ]:
X = generate_sample_ktensor()
X.double()  # Converts to an array.

## Use `tendiag` or `sptendiag` to convert a `ktensor` to a `ttensor`
A `ktensor` can be regarded as a `ttensor` with a diagonal core.

In [ ]:
X = generate_sample_ktensor()
R = len(X.weights)  # Number of factors in X.
core = ttb.tendiag(X.weights, ((R,) * X.ndims))  # Create a diagonal core.
Y = ttb.ttensor(core, X.factor_matrices)  # Assemble the ttensor
Y

In [ ]:
(X.full() - Y.full()).norm()  # They are the same.

In [ ]:
core = ttb.sptendiag(X.weights, ((R,) * X.ndims))  # Sparse diagonal core.
Y = ttb.ttensor(core, X.factor_matrices)  # Assemble the ttensor
Y

In [ ]:
(X.full() - Y.full()).norm()  # They are the same.

## Use `ndims` and `shape` for the dimensions of a `ktensor`

In [ ]:
X = generate_sample_ktensor()
X.ndims  # Number of dimensions.

In [ ]:
X.shape  # Tuple of the shapes.

In [ ]:
X.shape[1]  # Shape of the 2nd mode.

## Subscripted reference for a `ktensor`

In [ ]:
X = generate_sample_ktensor()
X.weights[1]  # Weight of the 2nd factor.

In [ ]:
X.factor_matrices[1]  # Extract a matrix.

## Subscripted assignment for a `ktensor`

In [ ]:
X = generate_sample_ktensor()
X.weights = np.ones(X.weights.shape)  # Insert new multipliers.
X

In [ ]:
X.weights[0] = 7  # Change a single element of weights.

In [ ]:
X.factor_matrices[2][:, [0]] = np.ones((2, 1))  # Change the matrix for mode 3.
X

## Using negative indexing for the last array index

In [ ]:
X = generate_sample_ktensor()
X.factor_matrices[0][-1:, :]
X.shape

In [ ]:
X = generate_sample_ktensor()
X.factor_matrices[0][0][
    1 : (np.prod(X.shape) - 1)
].item()  # Calculates factor_matrix[0][1]

## Adding and subtracting `ktensor`s
Adding two ktensors is the same as concatenating the matrices.

In [ ]:
np.random.seed(0)
X = ttb.ktensor(
    [np.random.rand(4, 2), np.random.rand(2, 2), np.random.rand(3, 2)]
)  # Data.
Y = ttb.ktensor(
    [np.random.rand(4, 2), np.random.rand(2, 2), np.random.rand(3, 2)]
)  # More data.
X, Y

In [ ]:
Z = X + Y  # Concatenates the factor matrices.
Z

In [ ]:
Z = X - Y  # Concatenates as with plus, but changes the weights.
Z

In [ ]:
(Z.full() - (X.full() - Y.full())).norm()  # Should be zero.

## Basic operations with a `ktensor`

In [ ]:
+X  # Calls uplus.

In [ ]:
-X  # Calls uminus.

In [ ]:
5 * X  # Calls mtimes.

## Use `permute` to reorder the modes of a `ktensor`

In [ ]:
np.random.seed(0)
X = ttb.ktensor(
    [np.random.rand(4, 2), np.random.rand(2, 2), np.random.rand(3, 2)]
)  # Data.
X.permute(np.array((1, 2, 0)))  # Reorders modes of X.

## Use `arrange` to normalize the factors of a `ktensor`
The function `arrange` normalizes the columns of the factors and then arranges the rank-one pieces in decreasing order of shape.

In [ ]:
np.random.seed(0)
X = ttb.ktensor(
    [np.random.rand(3, 2), np.random.rand(4, 2), np.random.rand(2, 2)]
)  # Unit weights.
X

In [ ]:
X.arrange()  # Normalized and rearranged.
X

## Use `fixsigns` for sign indeterminacies in a `ktensor`
The largest magnitude entry for each factor is changed to be positive provided that we can flip the signs of pairs of vectors in that rank-1 component.

In [ ]:
np.random.seed(0)
X = ttb.ktensor(
    [np.random.rand(4, 2), np.random.rand(2, 2), np.random.rand(3, 2)]
)  # Data.
Y = X
Y.factor_matrices[0][:, 0] = -Y.factor_matrices[0][
    :, 0
]  # switch the sign on a pair of columns
Y.factor_matrices[1][:, 0] = -Y.factor_matrices[1][:, 0]
Y

In [ ]:
Y.fixsigns()

## Use `ktensor` to store the 'skinny' SVD of a matrix

In [ ]:
np.random.seed(0)
A = np.random.rand(4, 3)
A

In [ ]:
[U, S, Vh] = np.linalg.svd(A, full_matrices=False)  # Compute the SVD.
# Numpy Expects U*S*Vh where pyttb expects U*S*V'
X = ttb.ktensor([U, Vh.transpose()], S)  # Store the SVD as a ktensor.
X

In [ ]:
print(f"U*S*Vh:\n{U @ np.diag(S) @ Vh}")
print(
    f"\nX.factor_matrices[0]@np.diag(X.weights)@(X.factor_matrices[1].transpose()):\n\
{X.factor_matrices[0] @ np.diag(X.weights) @ (X.factor_matrices[1].transpose())}"
)
print(f"\nX.full():\n{X.full()}")  # Reassemble the original matrix.

## Displaying a `ktensor`

In [ ]:
X = generate_sample_ktensor()
X

In [ ]:
print(X)